<a href="https://colab.research.google.com/github/twinstae/tripReviewAnalysisSystem/blob/master/%ED%81%AC%EB%A1%A4%EB%9F%AC-%EC%A0%84%EC%B2%98%EB%A6%AC/TRAS_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException,StaleElementReferenceException 
from selenium.webdriver.common.keys import Keys
import time
import re
import sys
from PyQt5.QtWidgets import *
from PyQt5 import uic

In [2]:
#키를 파일 이름으로 저장
def data_to_csv(dict_n, name):
    '''
    입력 : 딕셔너리 데이터
    출력 : csv파일 형태로 원시 자료 폴더에 출력.
    '''
    attraction = dict_n.keys
    address = dict_n.values
    data = [attraction, address]
    git_address = 'C:/Users/taehee/Documents/GitHub'
    attractions_df = pd.DataFrame(data)
    attractions_df.to_csv(git_address + '/tripReviewAnalysisSystem/크롤러-전처리/원시자료/' + name +'.csv', mode = 'w')

In [3]:
def reviews(sample_size):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 표본 리뷰의 숫자자.
    맥락 : 리뷰를 추출할 즐길거리 페이지에 있다.
    출력 : 한 즐길거리에 대한 리뷰를 모아놓은 데이터 프레임

    각 리뷰에서 새로운 항목을 추출하고 싶다면, columns와 for문을 모두 수정해야한다.
    '''
    reviews = []
    columns = ['star_point','title','text','Date of experience']
    #,'reviewer_address'
        
    #리뷰 크기를 텍스트로 찾아와, 콤마를 제거하여, 정수형으로 저장.
    review_size_text = driver.find_element_by_class_name('attractions-community-content-TabBarContent__tabCount--2hTdj').text
    review_size = int(re.sub(',','', str(review_size_text)))

    n = 5
    count = 0
    while n < sample_size:
        try:
            foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
            show_more = foot.find_element_by_tag_name('button')  
            driver.execute_script("arguments[0].click();", show_more)
            n += 5
        except:
            if count > 3:
                break
            count +=1
            time.sleep(2)
                
    review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')

    for review_card in review_cards:
        try:
            read_more = review_card.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            read_more.click()
        except:

            
            continue
    
    # 드라이버 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')
    
    for review_element in review_elements:
        try:
            review = [
                int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                review_element.find('a', class_='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT').find('span').find('span').text,
                review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text,
                review_element.find('span', class_='location-review-review-list-parts-EventDate__event_date--1epHa').contents[1]
                #,review_element.find('span', class_='social-member-common-MemberHometown__hometown--3kM9S').contents[1]
            ]
        except IndexError:
            #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
            continue
        except AttributeError:
            #주소를 공개하지 않는 에러, 무시하고 계속
            continue
        reviews.append(review)

    return pd.DataFrame(reviews, columns = columns)

In [4]:
def crawl_reviews():
    
    columns = ['title','text']
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    member_review = driver.find_element_by_class_name('pageNumbers')
    group_size = int(member_review.find_elements_by_tag_name('a')[-1].text)
    
    if group_size >= 60 :
        group_count = 60
    else :
        group_count = group_size
        
    review_count = 1
    reviews = []
    
    while review_count < group_count:
        
        review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')

        try:
            read_more = review_cards[0].find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            driver.execute_script("arguments[0].click();", read_more)
        except ElementNotInteractableException:
            print_qt("I already Click the read more buttons!", myWindow)
        except ElementClickInterceptedException:
            print_qt("I can't click the read more buttons!", myWindow)
        except StaleElementReferenceException:
            print_qt("I can't click the read more buttons!", myWindow)
            
            
        time.sleep(2)
        
        # 드라이버 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')

        for review_element in review_elements:
            try:
                review = [
                    int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                    review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text
                ]
            except IndexError:
                #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
                continue
            except AttributeError:
                #주소를 공개하지 않는 에러, 무시하고 계속
                continue
            reviews.append(review)

        review_count += 1

        str_review_count = str(review_count)

        member_next = member_review.find_element_by_link_text(str_review_count)
        driver.execute_script("arguments[0].click();", member_next)
        time.sleep(3)
    df = pd.DataFrame(reviews, columns = columns)

    return (df)

In [ ]:
def find_address():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contact = soup.find_all(class_ = 'attractions-contact-card-ContactCard__contactRow--3Ih6v')
    address_block = contact[0]
    address_spans = address_block.contents
    address_span = address_spans[1]
    address = address_span.text
    return address

"""data_test = [names, location]

df_list = pd.DataFrame(data_test, columns = columns)
df_list.to_csv('crawler test.csv')
"""

"data_test = [names, location]\n\ndf_list = pd.DataFrame(data_test, columns = columns)\ndf_list.to_csv('crawler test.csv')\n"

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
#해당 여행지의 페이지로 이동한 뒤에 호출
def crawl_attraction_by_types():
    '''
    현재 페이지에 있는 리뷰들을 여행자 타입별로 데이터프레임의 딕셔너리로 만들어 반환
    '''
    
    review_panel = driver.find_element_by_class_name('location-review-review-list-parts-ReviewFilters__filters_wrap--y1t86')    
    process1 = review_panel.find_elements_by_class_name('is-shown-at-tablet')
    checkbox_list = process1[0].find_elements_by_tag_name('label')   
    
    checkbox_dict = {checkbox.text: checkbox for checkbox in checkbox_list}
    attraction_dict = {}
    for traveler_type, check in checkbox_dict.items():
        print_qt(traveler_type, myWindow)
        check.click()
        data = crawl_reviews()
        attraction_dict[traveler_type] = data
        check.click()
    return attraction_dict

In [ ]:
def load_attraction(link, function):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''
    #새 창을 열고 링크로 간다.
    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    
    time.sleep(2)
    original_window = driver.current_window_handle
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    
    #즐길거리 별로 샘플사이즈를 어떻게 정할 것인지?    
    result = function()

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    return result
    #result.head()

In [ ]:
def load_page(start, end, function):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 크롤링할 페이지 내 시작 번호와 끝 번호
    출력 : 페이지의 리뷰들을 csv파일로 출력
    '''

    #모든 여행지의 리스트를 만든다
    is_first = True
    print_qt("I am in the load_page", myWindow)
    try:
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__container--3eHZU')
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        is_first = True
    except NoSuchElementException :
        main_page = driver.find_element_by_class_name('scrollAdMain')
        attraction_elements = main_page.find_elements_by_class_name('listing_info')
        is_first = False
    print_qt("I found the elements!", myWindow)

    attraction_names = []
    for attraction_element in attraction_elements[start:end]:      
        if is_first:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        else:
            div_class = attraction_element.find_element_by_class_name('tracking_attraction_title')
            link = div_class.find_element_by_tag_name('a')
            attraction_name = link.text
        attraction_names.append((attraction_name, link))   
    print_qtbl_textHere.append("I found the attraction_names", myWindow)
    
    file_name = str(start) +"~"+str(end)+"_"+ function.__name__
    
    attraction_dict = {}
    for attraction_name, link in attraction_names:
        if len(attraction_dict) < end:
            stime = time.time()
            
            # 여행지 크롤링
            attraction_dict[attraction_name] = load_attraction(link, function)
            
            print_qt(attraction_name,  " " + str(time.time() -stime) + "초 걸림 ", myWindow)

            value_type = type(list(attraction_dict.values())[0])
            if value_type == type('s'):
                if len(attraction_dict) % 5 == 0: 
                    data_to_csv(attraction_dict, name = file_name)
                    print_qt("저장했어요!", myWindow)
            if value_type == type({}):                  
                for attraction_name, reviews_by_type_dict in attraction_dict.items():                
                    for trav_type, type_df in reviews_by_type_dict.items():                     
                        file_name = attraction_name +'_'+trav_type+'_df'
                        git_address = r'C:\Users\Jeong/'                        
                        type_df.to_csv(git_address + file_name +'.csv', mode = 'w')
                print_qt("저장했어요!", myWindow)
        else:
            data_to_csv(attraction_dict, name = file_name)
            print_qt("저장했어요!", myWindow)
            break

In [ ]:
attraction_dict ={'taeheegung': 'name'}
value_type = type(list(attraction_dict.values())[0])
value_type == type('s')

True

In [ ]:
def func_is_first_page(driver):
    try:
        time.sleep(1)
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)
        is_first_page = True
        print_qt("it is first page!", myWindow)
    except NoSuchElementException:
        print_qt('continue!', myWindow)
        is_first_page = False
    except:
        print_qt("exception", myWindow)

In [ ]:
def get_start_page(soup, start, end):
    result_page = 1
    try:
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
    except NoSuchElementException:
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        
    if start // 30 > 0:
        if start // 30 > 6:
            target_num = 6
        else:
            target_num = (start // 30) + 1
        target_page = page_numbers.find_element_by_link_text(str(target_num))
        time.sleep(1)
        driver.execute_script("arguments[0].click();", target_page)
        time.sleep(1)
        result_page = target_num
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        is_first_page = False
    else:
        page_numbers_soup = soup.find("div", class_='attractions-attraction-overview-main-Pagination__container--PUXGq') 
        result_page = page_numbers_soup.find_all("span")[1].text
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        
    return int(result_page), page_numbers


In [ ]:
def crawl_pages(is_first_page, start, end, function):
    keep_going = True    
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    print_qt("I made a soup", myWindow)
    
    selected_page, page_numbers = get_start_page(soup, start, end)

    while keep_going:
        # 현재 페이지 번호 알아내기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print_qt("I made a soup", myWindow)
            
                # 시작점이 뒷 페이지면 지나간다
        if start <= selected_page * 30:
            strat_in_page = start-(selected_page-1)*30
            if strat_in_page < 0:
                strat_in_page = 0
            assert strat_in_page <= 30
            
            end_in_page = end-(selected_page-1)*30
            if end_in_page > 30:
                end_in_page = 30                
            assert 0 <= end_in_page
            if is_first_page:
                is_first_page = False
                
            load_page(strat_in_page, end_in_page, function)
        
        # 끝나는 지점을 지나갔으면 루프를 끝낸다.
        if (selected_page - 1) * 30 >= end:
            keep_going = False
            break
            
        print_qt("I find the current page", myWindow)

        # 다음 페이지로 넘어가기
        time.sleep(1)
        selected_page += 1
        next_page = page_numbers.find_element_by_link_text(str(selected_page))
        driver.execute_script("arguments[0].click();", next_page)
        

In [ ]:
def crawl_function(start, end, function):
    '''
    입력 : 크롤링할 즐길거리의 시작 번호와 끝 번호
    출력 : 해당 구간의 즐길거리 리뷰들csv로 출력
    '''    
        
    #처음 한 번만 see more을 누른다.
    is_first_page = func_is_first_page(driver)
    
    time.sleep(1)
    crawl_pages(is_first_page, start, end, function)

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("lang=en-US")
#이미지를 보지 않음
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver_address = 'C:/Users/taehee/Downloads/chromedriver_win32/chromedriver'

driver = webdriver.Chrome(driver_address, chrome_options=options)
wait = WebDriverWait(driver, timeout=2)
driver.implicitly_wait(5)
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')


C:\Users\taehee\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [ ]:
form_class = uic.loadUiType("lineeditTest.ui")[0]

class WindowClass(QMainWindow, form_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)

        #버튼에 기능을 할당하는 코드
        self.btn_changeText.clicked.connect(self.start_crawl)

    def start_crawl(self): 
        crawl_function(start= 1, end= 10, function = crawl_attraction_by_types)

In [ ]:
if __name__ == "__main__" :
    app = QApplication(sys.argv)
    myWindow = WindowClass()
    def print_qt(text, myWindow):
        WebDriverWait(driver, 10).until(
            myWindow.lbl_textHere.append(text)
        )
    myWindow.show()
    app.exec_()
    